# Comment créer une base de données vectorielle avec PostgreSLQ et pgvector

## Etapes

1. Créer la BDD PostgreSQL + pgvector dans un conteneur Docker
2. Se connecter avec psql ou pgAdmin (pour vérification)
3. Importer les données (textes)
4. Créer les vecteurs avec Sentence Transformers
5. Créer une table 'quora' dans la BDD
6. Sauvegarder les données et les vecteurs dans la BDD
7. Utiliser la recherche vectorielle 

## 🐳 Docker : Créer la BDD PostgreSQL + pgvector avec dans un conteneur 

### 1. `Docker-compose.yaml`

Tout d'abord, nous devons créer un fichier `docker-compose.yml` avec les services nécessaires.

Dans ce fichier, nous définissons un service appelé `db` qui est basé sur l'image Docker `pgvector/pgvector:pg16`. Le service expose le port `5432` pour interagir avec la base de données et configure des variables d'environnement pour le nom de la base de données, l'utilisateur, le mot de passe et la méthode d'authentification. De plus, nous montons un fichier `init.sql` dans le répertoire `/docker-entrypoint-initdb.d` à l'intérieur du conteneur à des fins d'initialisation.

### 2. Fichier `init.sql`

Dans ce script `init.sql`, nous activons l'extension `pgvector`, si elle n'existe pas déjà. Ensuite, nous créons une table appelée `embedding` avec les colonnes : `id`, `embedding`, `text` et `created_at`.

```
CREATE EXTENSION IF NOT EXISTS vector;

CREATE TABLE IF NOT EXISTS embeddings (
  id SERIAL PRIMARY KEY,
  embedding vector,
  text text,
  created_at timestamptz DEFAULT now()
);
```

### 3. Créer le conteneur

```
docker-compose up -d
```

Cette commande créera un conteneur Docker avec le serveur PostgreSQL et l'extension `pgvector` déjà installés et configurés, en fonction des spécifications du fichier `docker-compose.yml`.

## 🔌 Se connecter avec psql (dans le conteneur)

### Aller dans le conteneur

`docker exec -it <container id> bash`

### Se conncter à la base de donnée avec `psql`

`psql -h localhost -U testuser -d vectordb`

## 📚 Importation des bibliothèques Python

In [ ]:
# ! pip install sentence-transformers "psycopg[binary]" --quiet

In [1]:
import os
import time

import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
import psycopg
from pgvector.psycopg import register_vector

ModuleNotFoundError: No module named 'numpy'

## 3. Importer le jeu de données (textes)

Le jeu de donnée consiste en **11812 paires de question-réponses**, extraites du dataset **"piaf v1.2"**, disponible sur https://www.data.gouv.fr/. L'objectif est de créer des représentations vectorielles pour les questions avec un modèle **Sentence Transformers (sBert)**. Ainsi, nous pourrons effectuer une **recherche sémantique** dans les questions depuis une requète textuelle ou **trouver les questions les plus similaires** entre elles.

|  | Description |
| --- | --- |
| Nom du fichier | question-reponse-sans-texte.csv |
| Description du fichier | Extraction des questions-réponses à partir du fichier piaf-v1.2.json pour les visualiser au format CSV |
| Colonnes | 2 colonnes: ['question', 'reponse'] |
| Lignes | 11812 lignes | 
| Type MIME | text/csv |
| Créée le | 25 mars 2021 |
| Taille | 990.9Ko |

- https://www.data.gouv.fr/fr/datasets/piaf-le-dataset-francophone-de-questions-reponses/

In [2]:
# question-reponse-sans-texte.csv
dataset_url = "https://www.data.gouv.fr/fr/datasets/r/14159082-d1be-417e-a67c-c3c494c7a4ad"

dataset_df = pd.read_csv(dataset_url)

In [3]:
dataset_df.shape

(11812, 2)

In [4]:
corpus_questions = dataset_df.loc[:, 'question'].tolist()
corpus_reponses = dataset_df.loc[:, 'reponse'].tolist()

In [6]:
# Afficher les 10 premières paires de question-réponse.
for question, reponse in zip(corpus_questions[:10], corpus_reponses[:10]):
    print(f"{question} => {reponse}")

Quel architecte fût à l'origine des plans du Woolworth building? => Cass Gilbert
Où se trouvait Franck Woolworth lors de l'inauguration de son immeuble New Yorkais ? => Washington
Comment fût payé le bâtiment commandé par Franck Woolworth? => en cash
En quelle année ouvrit le Woolworth Building ? => 1913
Qui commanda la construction du Woolworth Building ? => Frank Woolworth
Quelle femme devint reine aux côtés de Philippe le Bel ? => Jeanne Ire de Navarre
Quel créancier du roi fut supprimé en 1312 ? => l'ordre du Temple
Quel créancier du roi fut supprimé en 1312 ? => l'ordre du Temple
Quelle raison pousse Philippe Le Bel à organiser les premiers Etats généraux ? => pour lever de nouveaux impôts
Quel souverain utilise les dévaluations monétaires pour s'enrichir ? => Philippe IV le Bel


## 4. Encoder les questions avec Sentence Transformers

**SentenceTransformers** est un framework Python pour la création de représentations vectorielles de phrases, de textes et d'images.

- https://www.sbert.net/index.html
- https://huggingface.co/dangvantuan/sentence-camembert-base

In [109]:
# Modèle Sentence Transformer pour l'encodage du jeu de données ou de la requète de recherche sémantique
# Modèle pré-entraîné d'encodage de textes en français
model_name = "dangvantuan/sentence-camembert-base" # vector size = 768
model = SentenceTransformer(model_name, device='cuda')

In [ ]:
questions_embeddings = model.encode(corpus_questions, show_progress_bar=True)

## 5. Sauvegarder les données et les vecteurs dans la BDD

- **Pyscopg3** : https://www.psycopg.org/psycopg3/docs/basic/usage.html

In [127]:
# DEFINE THE DATABASE CREDENTIALS
user = 'testuser'
password = 'testpwd'
host = 'localhost'
port = 5432
database = 'vectordb'

db_url = f"postgresql://{user}:{password}@{host}:{port}/{database}"

In [152]:
%%time
# Connect to an existing database
with psycopg.connect(conninfo=db_url) as conn:
    # Activation de l'extension pgvector (si elle n'existe pas déjà)
    conn.execute("CREATE EXTENSION IF NOT EXISTS vector;")
    # Enregistrez le type 'vecteur' avec votre connexion
    register_vector(conn)
    # Effacement de la table (si elle n'existe pas déjà)
    conn.execute("""DROP TABLE IF EXISTS piaf CASCADE;""")
    # Création de la table
    conn.execute("""
        CREATE TABLE IF NOT EXISTS piaf (
            id serial PRIMARY KEY,
            question text NOT NULL,
            reponse text NOT NULL,
            embedding vector(768) NOT NULL,
            created_at timestamptz DEFAULT now()
            );
        """)
    # Sauvegarde des questions/reponses et des vecteurs
    for question, reponse, embedding in zip(corpus_questions, corpus_reponses, questions_embeddings):
        conn.execute("""INSERT INTO piaf (question, reponse, embedding) VALUES (%s, %s, %s);""", (question, reponse, embedding))
    # AJouter un index (recherche approximative)
    conn.execute('CREATE INDEX IF NOT EXISTS idx_piaf_hnsw ON piaf USING hnsw (embedding vector_cosine_ops) WITH (m = 8, ef_construction = 24);')
    # Make the changes to the database persistent
    conn.commit()

CPU times: user 1.03 s, sys: 303 ms, total: 1.33 s
Wall time: 13.8 s


## 6. Utiliser la recherche vectorielle 

- https://huggingface.co/sentence-transformers/quora-distilbert-multilingual

### Rechercher par requète textuelle

In [129]:
%%time
query = "quelle est la date de la libération ?"

# Encoder la requète avec Sentence Transformers
embedding_query = model.encode(query)

CPU times: user 59.1 ms, sys: 0 ns, total: 59.1 ms
Wall time: 58.1 ms


In [132]:
%%time
# Connect to an existing database
with psycopg.connect(conninfo=db_url) as conn:
    # Enregistrez le type 'vecteur' avec votre connexion
    register_vector(conn)
    # Executer la commande
    res = conn.execute("""
        SELECT
            id,
            1 - (embedding <=> %s) AS cosine_similarity,
            question,
            reponse
        FROM piaf ORDER BY cosine_similarity DESC LIMIT 10;
    """, (embedding_query, )).fetchall()

    for row in res:
        print(f"id : {row[0]} | score : {round(row[1], 4)} | Question : {row[2]} | Réponse : {row[3]}")

id : 7495 | score : 0.7975 | Question : Quelle est la date de la libération de Paris ? | Réponse : 25 août 1944
id : 3762 | score : 0.7365 | Question : Quelle est la date de l’inauguration ? | Réponse : 11 octobre 1854
id : 3763 | score : 0.7365 | Question : Quelle est la date de l’inauguration ? | Réponse : le 11 octobre 1854
id : 10373 | score : 0.7156 | Question : A quelle date ? | Réponse : 28 septembre 1746
id : 2754 | score : 0.7 | Question : Quelles est la date de création de la réserve ? | Réponse : 1991
id : 2755 | score : 0.7 | Question : Quelles est la date de création de la réserve ? | Réponse : en 1991
id : 4156 | score : 0.6995 | Question : quelle est la date de la réforme? | Réponse : 1995
id : 4155 | score : 0.6995 | Question : quelle est la date de la réforme? | Réponse : 1995
id : 9578 | score : 0.6947 | Question : Quelle est sa date de sortie? | Réponse : 1995
id : 10037 | score : 0.6606 | Question : Quelle est la date de création du Comité? | Réponse : 18 décembre 2

In [134]:
# Retourner les résultats dans une DataFrame
def query_to_dataframe(query, column_names):

    # Connect to an existing database
    with psycopg.connect(conninfo=db_url) as conn:
        # Enregistrez le type 'vecteur' avec votre connexion
        register_vector(conn)
        embeddings_query = model.encode(query)
        # Execute a command: this creates a new table
        res = conn.execute("""
            SELECT
                id,
                1 - (embedding <=> %s) AS cosine_similarity,
                question,
                reponse
            FROM piaf ORDER BY cosine_similarity DESC LIMIT 10;
        """, (embeddings_query, )).fetchall()
        res_df = pd.DataFrame(res, columns=column_names)
        return res_df

In [137]:
%%time
# Cherchez une question en changeant cette requète
query = "Quelle est la date de la libération ?"
query_to_dataframe(query, ['id', 'cosine_similarity', 'question', 'reponse'])

CPU times: user 54.4 ms, sys: 0 ns, total: 54.4 ms
Wall time: 113 ms


,id,cosine_similarity,question,reponse
0,7495,0.809619,Quelle est la date de la libération de Paris ?,25 août 1944
1,3763,0.740143,Quelle est la date de l’inauguration ?,le 11 octobre 1854
2,3762,0.740143,Quelle est la date de l’inauguration ?,11 octobre 1854
3,10373,0.723416,A quelle date ?,28 septembre 1746
4,4155,0.707283,quelle est la date de la réforme?,1995
5,4156,0.707283,quelle est la date de la réforme?,1995
6,2754,0.698089,Quelles est la date de création de la réserve ?,1991
7,2755,0.698089,Quelles est la date de création de la réserve ?,en 1991
8,9578,0.695604,Quelle est sa date de sortie?,1995
9,10448,0.665852,Quelle date pour la bataille ?,24 mars 1793


### Les plus proches voisins d'une ligne (id aléatoire)

Résultats : 
- La première ligne est une ligne choisie au hasard dans la base de donnée.
- Les lignes suivantes sont les lignes les plus proches sémantiquement, par ordre décroissant de similarité cosinus.

In [148]:
import random
rand_id = random.randint(0,11812)

print(f"id aléatoire : {rand_id}")
print()

# Connect to an existing database
with psycopg.connect(conninfo=db_url) as conn:

    res = conn.execute("""
        SELECT
        id,
        1 - (embedding <=> (SELECT embedding FROM piaf WHERE id = %s)) AS cosine_similarity,
        question,
        reponse          
        FROM piaf
        ORDER BY cosine_similarity DESC LIMIT 20;
    """, (rand_id, )).fetchall()

    res_df = pd.DataFrame(res, columns=['id', 'cosine_similarity', 'question', 'reponse'])
    
res_df

id aléatoire : 289



,id,cosine_similarity,question,reponse
0,289,1.000000,Quel métier exerce Pierre-Jakez Hélias à Rennes ?,professeur
1,294,0.735688,Quel métier a exercé Pierre-Jakez Hélias au jo...,rédacteur en chef
2,297,0.733118,Avec qui étudie Pierre-Jakez Hélias à Rennes ?,Paul Ricœur et Pierre Hervé
3,298,0.733118,Avec qui étudie Pierre-Jakez Hélias à Rennes ?,Paul Ricœur et Pierre Hervé
4,290,0.668872,Quel événement historique marque un changement...,Libération
5,312,0.611864,Dans quel régiment a officié le père de Pierre...,35° d'artillerie
6,313,0.611864,Dans quel régiment a officié le père de Pierre...,au 35° d'artillerie
7,301,0.595106,Quel choix aurait pu faire Pierre-Jakez Hélias...,entrer dans la vie active
8,300,0.595106,Quel choix aurait pu faire Pierre-Jakez Hélias...,entrer dans la vie active
9,299,0.595106,Quel choix aurait pu faire Pierre-Jakez Hélias...,entrer dans la vie active
